In [ ]:
%matplotlib inline
### import libraries
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns   

In [ ]:
df_311 = pd.read_csv ('Datasets/311_Service_Requests_from_2010_to_Present.csv')

In [ ]:
# Task 1. . Understand the dataset:
#   1.1 Import the dataset
#   1.2 Visualize the dataset
#   1.3 Print the columns of the DataFrame 1.4 Identify the shape of the dataset
#   1.5 Identify the variables with null values

df_311.describe()


In [ ]:
df_311.head()

In [ ]:
df_311.info

In [ ]:
df_311.columns

In [ ]:
df_311.shape

In [ ]:
# Task 2. Perform basic data exploratory analysis:
#   2.1 Draw a frequency plot to show the number of null values in each column of the DataFrame
#   2.2 Missing value treatment
#   2.2.1 Remove the records whose Closed Date values are null
#    2.3 Analyze the date column, and remove entries that have an incorrect timeline
#   2.3.1 Calculate the time elapsed in closed and creation date
#   2.3.2 Convert the calculated date to seconds to get a better representation
#   2.3.3 View the descriptive statistics for the newly created column
#   2.3.4 Check the number of null values in the Complaint_Type and City columns
#   2.3.5 Impute the NA value with Unknown City
#   2.3.6 Draw a frequency plot for the complaints in each city
#   2.3.7 Create a scatter and hexbin plot of the concentration of complaints across Brooklyn

df_311.isnull().sum()

In [ ]:
# Task 2. Perform basic data exploratory analysis:
# 2.1 Draw a frequency plot to show the number of null values in each column of the DataFrame


def plot_task2(df_311: df_311):
    if df_311.isnull().sum().sum() != 0:
        na_df = (df_311.isnull().sum() / len(df_311)) * 100 
        #print(na_df)     
        na_df = na_df.drop(na_df[na_df == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio %' : na_df})
        missing_data.plot(kind = "barh")
        plt.show()
    else:
        print('No NAs found')
plot_task2(df_311)


In [ ]:
# Task 2.2 Missing value treatment
# 2.2.1 Remove the records whose Closed Date values are null
df_cd_nul = df_311[df_311['Closed Date'].notnull()]

df_cd_nul["Closed Date"].isnull().sum()

In [ ]:
#  2.3 Analyze the date column, and remove entries that have an incorrect timeline
#  2.3.1 Calculate the time elapsed in closed and creation date
#  2.3.2 Convert the calculated date to seconds to get a better representation

import datetime

df_time = pd.read_csv ('Datasets/311_Service_Requests_from_2010_to_Present.csv', parse_dates=['Created Date','Closed Date'])

df_time['timeElapsed'] = df_time['Closed Date'] - df_time['Created Date']

#print(df['timeElapsed'])

df_time['seconds'] = df_time['timeElapsed'].dt.total_seconds()
print(df_time['seconds'])


In [ ]:
# 2.3.3 View the descriptive statistics for the newly created column
print(df_time['seconds'].describe().apply(lambda x: format(x, 'f')))


In [ ]:
# 2.3.4 Check the number of null values in the Complaint_Type and City columns
print("Number Complaint Type with null value = ", df_311["Complaint Type"].isnull().sum())
print("Number City with null value = ", df_311['City'].isnull().sum())
#print("Number City with NA value = ", df_311['City'].isna().sum())

In [ ]:
# 2.3.5 Impute the NA value with Unknown City

df_311['City'].fillna("Unknown City", inplace=True)
print("Number City with NA value = ", df_311['City'].isna().sum())

In [ ]:
# 2.3.6 Draw a frequency plot for the complaints in each city
plt.figure(figsize=(8,8))
plt.title('Total complaints')
sns.countplot(y='City',data=df_311)
plt.show()

In [ ]:
# 2.3.7 A Create a scatter and hexbin plot of the concentration of complaints across Brooklyn
df_311[['Longitude', 'Latitude']].plot(kind = 'scatter', x='Longitude', y='Latitude', title = 'Complaints Concentration in Brooklyn', figsize = (10, 8))

In [ ]:
# Task 2.3.7 B
df_311[['Longitude', 'Latitude']].plot(kind = 'hexbin', x='Longitude', y='Latitude', gridsize=40,
    colormap = 'jet', mincnt=1, title = 'Complaints Concentration in Brooklyn', figsize = (10, 8))

In [ ]:
# Task 3 Find major types of complaints:
# Task 3.1 Plot a bar graph to show the types of complaints

plt.figure(figsize=(8,8))
plt.title('Complaints Types Distribution')
sns.countplot(y='Complaint Type', data=df_311)
plt.show()



In [ ]:
# Task 3.2 Check the frequency of various types of complaints for New York City
df_ny = df_311.loc[df_311['City'] == 'NEW YORK']
#df_ny['Complaint Type'].value_counts().head(21).plot(kind='bar', title = 'Complaint Frequency in New York')
print(df_ny.describe())
plt.figure(figsize=(8,8))
plt.title('Complaint Frequency in New York')
sns.countplot(y='Complaint Type', data=df_ny)
plt.show()


In [ ]:
# Task 3.3 Find the top 10 complaint types
df_top10 = df_311['Complaint Type'].value_counts().head(10)
print(df_top10)
df_top10.plot(kind='bar',figsize=(5,5))

In [ ]:
# Task 3.4 Display the various types of complaints in each city
df1 = pd.crosstab(df_311['City'],df_311['Complaint Type'])

df1.plot(kind='bar',stacked=True,figsize=(16,8))
plt.title('Complaints per city')
plt.show()

In [ ]:
# Task 3.5 Create a DataFrame, df_new, which contains cities as columns and complaint types in rows
df_new = pd.crosstab(df_311['Complaint Type'],df_311['City'])
df_new.head()

In [ ]:
#4. Visualize the major types of complaints in each city
    #4.1 Draw another chart that shows the types of complaints in each city in a single chart, where different colors show the different types of complaints
df1.plot(kind='bar',stacked=True,figsize=(16,8))
plt.title('Complaints per city')
plt.show()

In [ ]:
# 4.2 Sort the complaint types based on the average Request_Closing_Time grouping them for different locations
# First We need to create a new TimeDelta Columns as 'Request_Closing_Time' which will be the difference of 'Closed Date' and 'Created Date'.
exclude_columns = ['Created Date','Closed Date']

for col in exclude_columns:
    df_311[col] = pd.to_datetime(df_311[col],format='%m/%d/%Y %I:%M:%S %p')
    
for col in df_311.columns:
    if df_311[col].nunique() < 300 and col not in exclude_columns:
        df_311[col] = df_311[col].astype('category')
        


#print(df['timeElapsed'])

# filtering the rows where Complaint Type is Ferry Complaint
df_Search= df_311[df_311['Complaint Type'].str.contains('Ferry Complaint')]
#print(df_Search)

# convert date to seconds
df_311['Request_Closing_Time'] = (df_time['Closed Date'] - df_time['Created Date'])
df_311['Request_Closing_Time'] = df_311['Request_Closing_Time'].dt.total_seconds()
#print(df_311['Request_Closing_Time'])
#df_311['Request_Closing_Time'].head()

# Dropping the Unspecified Borough
df_311 = df_311[df_311["Borough"].str.contains("Unspecified") == False]

df_CT = df_311.groupby(['Complaint Type','Borough'])['Request_Closing_Time'].mean()
df_CT = df_CT.sort_values(axis=0, ascending=False, inplace=False, kind='quicksort', na_position='first')

df_CT = df_CT.dropna()
# suppress scientific notation 
#pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(df_CT)


In [ ]:
#5. See whether the average response time across different complaint types is similar (overall)
    #5.1 Visualize the average of Request_Closing_Time

df_CT = df_CT/1000
CT_plot = df_CT.plot(kind='bar', figsize=(20,7))
CT_plot.set_xlabel("Complaint Type")
CT_plot.set_ylabel("Request Closing Time X 1000 (Seconds)")

In [ ]:
# 6. Identify the significant variables by performing statistical analysis using p-values
#7. Perform a Kruskal-Wallis H test
#7.1 Fail to reject H0: All sample distributions are equal
#7.2 Reject H0: One or more sample distributions are not equal

from scipy import stats

 # Storing mean response time for various complaint types
complaints = df_311['Complaint Type'].value_counts().index 
for i in range(len(complaints)):
    exec("sample{} = df_311.loc[(df_311['Complaint Type'] == '{}') , 'Request_Closing_Time']".format(i+1 ,complaints[i]))
   
   
#for i in range(len(complaints)):
    #exec("sample{} = df_311.loc[(df_311['Complaint Type'] == '{}') , 'Request_Closing_Time']".format(i+1 ,complaints[i]))


print (stats.kruskal(sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample9,sample10,sample11,sample12,sample13,sample14,sample15,sample16,sample17,sample18,sample19,sample20,sample21, nan_policy='omit'))

print ("\n Assuming testing at Confidence level(95%) => alpha value = 0.05 \n pvalue (0.0) < alpha value (0.05) There is some significant relation between type of complaint and location (i.e) The type of complaint or service requested and the location are related")

